In [1]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

word_to_idx = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
print(word_to_idx)

{'idea': 15, 'en': 3, 'get': 20, 'gusta': 1, 'to': 8, 'la': 4, 'buena': 14, 'not': 17, 'on': 25, 'lost': 21, 'comer': 2, 'it': 7, 'No': 9, 'a': 18, 'una': 13, 'creo': 10, 'is': 16, 'good': 19, 'sea': 12, 'que': 11, 'si': 24, 'cafeteria': 5, 'Yo': 23, 'Give': 6, 'me': 0, 'at': 22}


In [10]:
VOCAB_SIZE = len(word_to_idx)
NUM_LABELS = 2

# Define the network!
class BoWClassifier(nn.Module): # Bag of Words classifier
    
    def __init__(self, num_labels, vocab_size):
        super(BoWClassifier, self).__init__()
        
        self.linear = nn.Linear(vocab_size, num_labels)
        
    def forward(self, x):
        # Pass the input through the linear layer,
        # then pass that through log softmax.
        return F.log_softmax(self.linear(x), dim = 1)
    
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [15]:
def make_bow_vec(sentence, word_to_idx):
    vec = torch.zeros(len(word_to_idx))
    for word in sentence:
        vec[word_to_idx[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_idx):
    return torch.LongTensor([label_to_idx[label]])

for param in model.parameters():
    print(param)

sample = data[0]
bow_vector = make_bow_vec(sample[0], word_to_idx)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

Parameter containing:

Columns 0 to 9 
-0.1832  0.1084 -0.0210  0.0315  0.1633  0.1552 -0.0797  0.0758  0.0702 -0.1550
 0.1824 -0.1032 -0.1285 -0.1198  0.1159  0.0822  0.0065  0.1608  0.0997 -0.0172

Columns 10 to 19 
-0.0414 -0.1456 -0.1397  0.1197 -0.1248  0.0863  0.0162 -0.0046 -0.0354  0.1476
-0.0324 -0.1911  0.0073  0.1543 -0.0623  0.0647  0.1316 -0.1191  0.1890 -0.1014

Columns 20 to 25 
 0.0873 -0.0912 -0.1551 -0.1111 -0.1305 -0.1720
-0.1501  0.0445  0.1873 -0.1707  0.0412 -0.0624
[torch.FloatTensor of size 2x26]

Parameter containing:
1.00000e-02 *
  0.5819
  3.0827
[torch.FloatTensor of size 2]

Variable containing:
-0.5980 -0.7983
[torch.FloatTensor of size 1x2]



In [16]:
label_to_idx = {"SPANISH": 0, "ENGLISH": 1}

for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vec(instance, word_to_idx))
    log_probs = model(bow_vec)
    print(log_probs)

Variable containing:
-0.7447 -0.6441
[torch.FloatTensor of size 1x2]

Variable containing:
-1.1964 -0.3600
[torch.FloatTensor of size 1x2]



In [17]:
# Define loss and optmization functions
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

# Training the model
for epoch in range(100):
    for instance, label in data:
        # zero out previous gradients
        model.zero_grad()
        
        bow_vec = autograd.Variable(make_bow_vec(instance, word_to_idx))
        target = autograd.Variable(make_target(label, label_to_idx))
        
        log_probs = model(bow_vec)
        
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vec(instance, word_to_idx))
    log_probs = model(bow_vec)
    print(log_probs)
    
print(next(model.parameters())[:, word_to_idx['creo']])

Variable containing:
-0.1317 -2.0924
[torch.FloatTensor of size 1x2]

Variable containing:
-3.1567 -0.0435
[torch.FloatTensor of size 1x2]

Variable containing:
 0.4042
-0.4780
[torch.FloatTensor of size 2]

